In [ ]:
%run "0a_Workspace_setup.ipynb"

from NHM_helpers.NHM_helpers import (
    hrus_by_poi,
    hrus_by_seg,
    subset_stream_network,
    create_poi_group,
)
from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals

from NHM_helpers.NHM_output_visualization import (
    retrieve_hru_output_info,
    create_sum_var_dataarrays,
    create_mean_var_dataarrays,
    create_sum_var_annual_gdf,
    create_sum_var_annual_df,
    create_sum_var_monthly_df,
    create_var_daily_df,
    create_var_ts_for_poi_basin_df,
)
from NHM_helpers.output_plots import *

from ast import literal_eval

poi_id_sel = None

crs = 4326

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf = create_hru_gdf(
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

### Retrieve pywatershed output file information.
> explain

In [ ]:
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

#### &#x270D;<font color='green'>**Enter Information:**</font> **Select HRU **output variable** and a **year** to map..
> The default is set to "Recharge," "mean_annual" with no gage slected. If a gage number is selected,
> the map will zoom to that location.

In [ ]:
# Create selection widgets

from ipywidgets import widgets
from ipywidgets import Text, HBox, VBox, Box
from IPython.display import display

##### Varibale selection widget
output_var_sel = output_var_list[
    8
]  # Set a default value so that the notebook will run without selection

style_date = {"description_width": "initial"}

style_var = {"description_width": "initial"}

v = widgets.Dropdown(
    options=output_var_list,
    value=output_var_list[1],
    description="Output variable for map and plot:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global output_var_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        output_var_sel = v.value
        # sel_flag = True


v.observe(on_change)
# display(v)

##### Year selection widget
list_of_years = year_list.copy()
list_of_years.append(
    "mean_annual"
)  # Append the mean annual so that the default will not be a year
sel_year = list_of_years[
    -1
]  # Set a default value so that the notebook will run without selection

yr = widgets.Dropdown(
    options=list_of_years,
    value=list_of_years[-1],
    description="Time step (year) for map:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global sel_year  # Have to set the var as global so that it is carried outside of the fucntion to the notebook
    if change["type"] == "change" and change["name"] == "value":
        sel_year = yr.value


yr.observe(on_change)

# #################################################
v2 = widgets.Combobox(
    # value=poi_df.poi_id.tolist()[0],
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change2(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v2.value


v2.observe(on_change2)

display(VBox([v, yr, v2]))

#### &#x1F6D1;If new selections are made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

In [ ]:
# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

### Read mapping elements

In [ ]:
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(
    hru_gdf, poi_df, poi_id_sel
)

In [ ]:
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)

In [ ]:
minimap = create_minimap()

### Create Map

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl
from folium.utilities import Element
from folium.plugins import FloatImage
import base64


# Create a geoDataFrame of the output variable's annual values to map
gdf_output_var_annual, value_min, value_max, var_units, var_desc = (
    create_sum_var_annual_gdf(
        out_dir,
        output_var_sel,
        plot_start_date,
        plot_end_date,
        water_years,
        hru_gdf,
        year_list,
    )
)

# Load standard map settings
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(
    hru_gdf, poi_df, poi_id_sel
)

# Clear map if previously created
m3 = folium.Map()

# Create map
m3 = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)

# Add base map layers
USGStopo_layer.add_to(m3)
OpenTopoMap.add_to(m3)
Esri_WorldImagery.add_to(m3)

# Add widgets
m3.add_child(minimap)
m3.add_child(MeasureControl(position="bottomright"))

# Create/Add scale bar and hru map object
hru_map, val_bar_image, value_min, value_max, same_value, color_bar = (
    create_annual_output_var_map(
        gdf_output_var_annual,
        output_var_sel,
        sel_year,
        var_units,
    )
)
hru_map.add_to(m3)

# Create/Add hru labels
marker_cluster_label_hru = create_hru_label(hru_gdf, cluster_zoom)
marker_cluster_label_hru.add_to(m3)

# Create/Add segment map
seg_map = create_segment_map_hide(seg_gdf)
seg_map.add_to(m3)

# Create/Add pois
poi_marker_cluster, poi_marker_cluster_label = create_poi_marker_cluster(
    poi_df, cluster_zoom
)
poi_marker_cluster.add_to(m3)
poi_marker_cluster_label.add_to(m3)

plugins.Fullscreen(position="topleft").add_to(m3)
folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

if not color_bar:
    scale_bar_txt = f"All {output_var_sel} values are {same_value} in the model domain. No value scale bar rendered."
    fig = None  # fig, ax = plt.subplots(figsize=(18, 0.5))
else:
    scale_bar_txt = f"Values for {output_var_sel} range from {value_min} to {value_max} {var_units} in the model domain."

title_html = f"<h1 style='position:absolute;z-index:100000;font-size: 28px;left:26vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '><strong>NHM {sel_year} {output_var_sel}</strong><br><h1 style='position:absolute;z-index:100000;font-size: 20px;left:31vw;right:5vw; top:4vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '> {var_desc}. {scale_bar_txt}</h1>"
# title_html = f"<h1 align='center' style='font-size: 14px;'><strong>NHM simulated {sel_year} {output_var_sel}, {var_units}</strong><br><h1 align='center' style='font-size: 14px;'> {var_desc}. {scale_bar_txt}</h1>"
m3.get_root().html.add_child(Element(title_html))

m3.add_child(val_bar_image)

m3.save(f"{output_var_sel}_{sel_year}_map.html")
m3

## Paste the poi_id in the field below

In [ ]:
v3 = widgets.Combobox(
    value=poi_id_sel,
    placeholder="Select Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change3(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v3.value


v3.observe(on_change3)

display(v3)

#### &#x1F6D1;If new selections are made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

### Create an interactive plot to evaluate an NHM ouput variable for poi basin.
> A plot is created for the output variable selected in this notebook that shows a cummulative value for the selected poi basin at the annual, monthly, and daily time steps. Individual poi basin hru contributions can also be compared to the poi basin values. The plot is interactive and can be saved using the widgets in the upper right-hand corner of the plot.

In [ ]:
fig_hru_sum_vars_for_poi = make_plot_var_for_hrus_in_poi_basin(
    out_dir,
    param_filename,
    water_years,
    hru_gdf,
    poi_df,
    output_var_sel,
    var_units,
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    plot_colors,
)

fig_hru_sum_vars_for_poi.show()

### Create an interactive plot to evaluate all NHM ouput variables as average fluxes (cubic feet/sec) for a poi basin.
> A plot is created for all listed output variables that shows average fluxes (cubic feet/sec) for the selected poi basin at the annual, monthly, and daily time steps. Individual output variables can be added and removed. The plot is interactive and can be saved using the widgets in the upper right-hand corner of the plot.

In [ ]:
fig_var_fluxes_for_poi = oopla(
    out_dir,
    param_filename,
    water_years,
    hru_gdf,
    poi_df,
    output_var_list,
    output_var_sel,
    var_units,
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    plot_colors,
    var_colors_dict,
    leg_only_dict,
)

fig_var_fluxes_for_poi.show()